In [3]:
import json
import pandas as pd
import os
import shutil

DATA_DIR = "../data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")

store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도
    "longitude",  # 음식점 경도
    "review_cnt", # 리뷰 개수
    "category",  # 음식점 카테고리
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
    "gender",  # 성별
    "born_year",  # 태어난 해 
)

bhour_columns = (
    "store", # 음식점 고유번호
    "type",
    "week_type",
    "mon",
    "tue",
    "wed",
    "thu",
    "fri",
    "sat",
    "sun",
    "start_time",
    "end_time",
    "etc",
)

menu_columns = (
    "store", # 음식점 고유번호
    "menu",
    "price",
)

def import_data(data_path=DATA_FILE):
    """
    Req. 1-1-1 음식점 데이터 파일을 읽어서 Pandas DataFrame 형태로 저장합니다
    """

    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    stores = []  # 음식점 테이블
    reviews = []  # 리뷰 테이블
    bhours = [] # 영업시간
    menus = [] # 메뉴

    for d in data:
        # 음식점 정보
        categories = [c["category"] for c in d["category_list"]]
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                d["review_cnt"],
                "|".join(categories),

            ]
        )

        # 영업시간
        b = d["bhour_list"]
        if b:
            for bhour in b:
                bhours.append(
                    [d["id"], bhour["type"], bhour["week_type"], bhour["mon"], bhour["tue"], bhour["wed"], bhour["thu"], bhour["fri"], bhour["sat"], bhour["sun"], bhour["start_time"], bhour["end_time"], bhour["etc"]]
                )       

        # 리뷰
        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"], u["gender"], u["born_year"]]
            )

        # 메뉴
        m = d["menu_list"]
        if m:
            for menu in m:
                menus.append(
                    [d["id"], menu["menu"], menu["price"]]
                )


    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    bhour_frame = pd.DataFrame(data=bhours, columns=bhour_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_columns)

    return {"stores": store_frame, "reviews": review_frame, "bhours": bhour_frame, "menus": menu_frame}


def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)


def load_dataframes():
    return pd.read_pickle(DUMP_FILE)


def main():

    print("[*] Parsing data...")
    data = import_data()
    print("[+] Done")

    print("[*] Dumping data...")
    dump_dataframes(data)
    print("[+] Done\n")

    data = load_dataframes()

    term_w = shutil.get_terminal_size()[0] - 1
    separater = "-" * term_w

    print("[음식점]")
    print(f"{separater}\n")
    print(data["stores"].head())
    print(f"\n{separater}\n\n")

    print("[리뷰]")
    print(f"{separater}\n")
    print(data["reviews"].head())
    print(f"\n{separater}\n\n")

    print("[영업시간]")
    print(f"{separater}\n")
    print(data["bhours"].head())
    print(f"\n{separater}\n\n")

    print("[메뉴]")
    print(f"{separater}\n")
    print(data["menus"].head())
    print(f"\n{separater}\n\n")

if __name__ == "__main__":
    main()

[*] Parsing data...
[+] Done
[*] Dumping data...
[+] Done

[음식점]
-----------------------------------------------------------------------------------------------------------------------

    id  store_name      branch     area           tel  \
0  152  101번지남산돈까스      안녕인사동점      인사동  02-6954-2884   
1  154  101번지남산돈까스  현대시티아울렛가산점  가산디지털단지  02-2136-9958   
2  411      135튀김집         새롬점      세종시  044-864-5252   
3  540        171+         신촌점      신촌역  02-6273-8552   
4  717    1984 나폴리          본점       서면  051-816-1984   

                         address   latitude   longitude  review_cnt  \
0           서울시 종로구 관훈동 155-2 2층  37.574554  126.983794           1   
1  서울특별시 금천구 가산동 60-8 현대시티아울렛 7층  37.477600  126.889000           2   
2             세종특별자치시 새롬동 341-74  36.484474  127.252608           1   
3           서울특별시 서대문구 대현동 27-33  37.558800  126.943000           1   
4          부산광역시 부산진구 전포동 687-18  35.155170  129.064134           1   

                                            

In [29]:
import requests
import json
import os
import shutil

days = [1, 2, 3, 4, 5, 6, 7]  # 월~일, 완결
result = []
for day in days:

    for page_num in range(0, 100):
        webtoon_list_url = "https://api2-page.kakao.com/api/v2/store/day_of_week_top/list?category=10&subcategory=0&page={}&bm=A&day={}".format(page_num, day)
        webtoon_req = requests.get(webtoon_list_url)

        if webtoon_req.status_code == 200:
            data = json.loads(webtoon_req.text)

            for item in data["list"]:
                if item["rank"]:
                    parsed_item = dict()
                    parsed_item["origin_id"] = item["series_id"]
                    parsed_item["title"] = item["title"]
                    parsed_item['description'] = item.get('caption', '')
                    parsed_item["genres"] = [item['sub_category_title']]
                    parsed_item["artists"] = item['author'].split(',')
                    parsed_item["link"] = 'https://page.kakao.com/home?seriesId={}'.format(item["series_id"])
                    parsed_item["rating"] = item.get("rating")

                    # 이미지 일단 제일 큰걸로 가져오기
                    parsed_item['thumb_img'] = "https://dn-img-page.kakao.com/download/resource?kid={}&filename=th1".format(item['land_thumb_img'])
                    parsed_item["distinct"] = "2"
                    result.append(parsed_item)

            if data['is_end']:
                break

with open('kakao_webtoon.json', 'w', encoding='UTF-8-sig') as file:
    file.write(json.dumps(result, ensure_ascii=False))

# print(webtoon_list_url)

In [20]:
import json
import pandas as pd
import os
import shutil

DATA_DIR = "../data"
DATA_FILE = os.path.join(DATA_DIR, "kakao_webtoon.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")


webtoon_columns = (
    "origin_id",  # 웹툰 고유번호
    "title",  # 웹툰 제목
    "description",  # 웹툰 설명
    "genres",  # 웹툰 장르
    "artists",  # 작가
    "rating",  # 평점
    "link",  # 웹툰 주소
    "thumb_img",  # 썸네일 이미지 주소
    "distinct", # 플랫폼 번호( 1: 네이버, 2: 카카오, 3: 다음)
)

# review_columns = (
#     "id",  # 리뷰 고유번호
#     "store",  # 음식점 고유번호
#     "user",  # 유저 고유번호
#     "score",  # 평점
#     "content",  # 리뷰 내용
#     "reg_time",  # 리뷰 등록 시간
#     "gender",  # 성별
#     "born_year",  # 태어난 해 
# )

def import_data(data_path=DATA_FILE):
    try:
        with open(data_path, encoding="utf-8-sig") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    webtoon = []  # 음식점 테이블
    reviews = []  # 리뷰 테이블
    bhours = [] # 영업시간
    menus = [] # 메뉴

    for d in data:
        # 음식점 정보
#         categories = [c["category"] for c in d["category_list"]]
        webtoon.append(
            [
                d["origin_id"],
                d["title"],
                d["description"],
                d["genres"],
                d["artists"],
                d["rating"],
                d["link"],
                d["thumb_img"],
                d["distinct"],

            ]
        )
        # 리뷰
#         for review in d["review_list"]:
#             r = review["review_info"]
#             u = review["writer_info"]

#             reviews.append(
#                 [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"], u["gender"], u["born_year"]]
#             )


    webtoon_frame = pd.DataFrame(data=webtoon, columns=webtoon_columns)
#     review_frame = pd.DataFrame(data=reviews, columns=review_columns)

    return {"webtoon": webtoon_frame}
#     return {"stores": store_frame, "reviews": review_frame }


def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)


def load_dataframes():
    return pd.read_pickle(DUMP_FILE)


def main():

    print("[*] Parsing data...")
    data = import_data()
    print("[+] Done")

    print("[*] Dumping data...")
    dump_dataframes(data)
    print("[+] Done\n")

    data = load_dataframes()

    term_w = shutil.get_terminal_size()[0] - 1
    separater = "-" * term_w

    print("[웹툰]")
    print(f"{separater}\n")
    print(data["webtoon"].head())
    print(f"\n{separater}\n\n")

#     print("[리뷰]")
#     print(f"{separater}\n")
#     print(data["reviews"].head())
#     print(f"\n{separater}\n\n")
if __name__ == "__main__":
    main()

[*] Parsing data...
[+] Done
[*] Dumping data...
[+] Done

[웹툰]
-----------------------------------------------------------------------------------------------------------------------

   origin_id      title             description    genres  \
0   47118512      롱리브더킹       대통령 꿈을 이룰 수 있을까!?   [드라마만화]   
1   55152866       불패검선           산수화에 담긴 절대무공!  [액션무협만화]   
2   48511988  독고3 [완전판]         모두가 기다려온 독고의 귀환    [소년만화]   
3   51703113       환골탈태        이 귀여운 생명체는 무엇..?   [드라마만화]   
4   53462628       후궁계약  후궁으로 입궁해 다오. 그게 내 의뢰다.    [순정만화]   

               artists  rating                                           link  \
0              [버드나무숲]     0.0  https://page.kakao.com/home?seriesId=47118512   
1  [FIST CAT, 김찬영, 적하]     0.0  https://page.kakao.com/home?seriesId=55152866   
2       [민(Meen), 백승훈]     0.0  https://page.kakao.com/home?seriesId=48511988   
3           [마고(mago)]     0.0  https://page.kakao.com/home?seriesId=51703113   
4       [밥꾹, 붉은마녀, 레죵]     0.0  https://pag